In [1]:
BASE_DIR = "/Users/denis/MyProjects/Summarization"
GRAPH_BASE_DIR = "/Users/denis/MyProjects/uncased_L-12_H-768_A-12"

In [2]:
# from importlib import reload
# from IPython.lib.deepreload import reload
# reload(tree_rank_summaraiser)

In [3]:
import os
import numpy as np
from bert_experimental.feature_extraction.bert_feature_extractor import BERTFeatureExtractor
from tree_rank_summaraiser import TreeRankSummariser
from data_generators.article_reference_yml_gen import ArticleReferenceGenerator
import global_const as glob_cont
import rouge
from tqdm import tqdm
from utils import evaluate

In [5]:
GRAPH_PATH = os.path.join(GRAPH_BASE_DIR, "model_for_inference.pbtxt")
VOCAB_PATH = os.path.join(GRAPH_BASE_DIR, "vocab.txt")

encoder = BERTFeatureExtractor(GRAPH_PATH, VOCAB_PATH)
summariser = TreeRankSummariser(4, encoder)

2020-05-02 19:25:31,791 :  From /Users/denis/anaconda3/envs/Summarization/lib/python3.6/site-packages/bert_experimental/feature_extraction/text_preprocessing.py:287: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

2020-05-02 19:25:31,895 :  From /Users/denis/anaconda3/envs/Summarization/lib/python3.6/site-packages/bert_experimental/feature_extraction/bert_feature_extractor.py:52: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

2020-05-02 19:25:31,896 :  From /Users/denis/anaconda3/envs/Summarization/lib/python3.6/site-packages/bert_experimental/feature_extraction/bert_feature_extractor.py:38: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2020-05-02 19:25:31,911 :  Using temporary folder as model directory: /var/folders/lx/02q9j0f52ds3f_h7jnq_dvmm0000gp/T/tmp725iecfv
2020-05-02 19:25:31,913 :  Using config: {'_model_dir': '/var/folders/lx/02q9j0f52ds3f_h7jnq_dvmm0000gp/T/tmp725iecfv', '_tf_

In [6]:
source = os.path.join(BASE_DIR, "data/yaml_format/article_summary_dataset.yml")
generator = ArticleReferenceGenerator(source)

In [7]:
rouge_metric = rouge.Rouge(metrics=['rouge-n', 'rouge-l', 'rouge-w'],
                           max_n=4,
                           limit_length=False,
                           length_limit_type='words',
                           apply_avg=True,
                           # apply_best=apply_best,
                           stemming=True)


In [8]:
all_hypothesis = []
all_references = []

count = 0
for sample_dict in tqdm(generator, total=generator.shape[0]):
    article_str = sample_dict[glob_cont.TF_RECORD_ARTICLE]
    summary_str = sample_dict[glob_cont.TF_RECORD_SUMMARY]
    label_str = sample_dict[glob_cont.TF_RECORD_LABEL]
    article_sum_list = summariser.get_summary(article_str)
    generated_sum = ". ".join(article_sum_list)
    
    all_hypothesis.append(summary_str)
    all_references.append(generated_sum)
    
    if count % 5 == 0:
        evaluate(rouge_metric, all_hypothesis, all_references)
        count += 1
    else:
        count += 1
    summary_path = os.path.join(BASE_DIR, f"output/generated_summary/{label_str}")
    with open(summary_path, mode="w") as file:
        for s in article_sum_list:
            file.write(s)
            file.write("\n")

evaluate(rouge_metric, all_hypothesis, all_references)

  1%|          | 25/2225 [00:29<42:46,  1.17s/it]


Meteor score: 0.8021724444444445
	rouge-1:	P: 65.43	R: 97.25	F1: 78.23
	rouge-2:	P: 62.11	R: 92.59	F1: 74.35
	rouge-3:	P: 60.00	R: 89.72	F1: 71.91
	rouge-4:	P: 57.86	R: 86.79	F1: 69.43
	rouge-l:	P: 33.95	R: 50.46	F1: 40.59
	rouge-w:	P: 33.95	R: 50.46	F1: 40.59
Meteor score: 0.4549952126554722
	rouge-1:	P: 45.82	R: 63.43	F1: 50.91
	rouge-2:	P: 34.51	R: 46.11	F1: 38.12
	rouge-3:	P: 31.27	R: 41.90	F1: 34.66
	rouge-4:	P: 29.52	R: 39.76	F1: 32.82
	rouge-l:	P: 30.37	R: 42.20	F1: 33.78
	rouge-w:	P: 30.37	R: 42.20	F1: 33.78
Meteor score: 0.4682315613173675
	rouge-1:	P: 49.78	R: 63.79	F1: 54.13
	rouge-2:	P: 37.60	R: 46.66	F1: 40.48
	rouge-3:	P: 34.47	R: 42.82	F1: 37.16
	rouge-4:	P: 32.77	R: 40.77	F1: 35.35
	rouge-l:	P: 33.50	R: 42.78	F1: 36.37
	rouge-w:	P: 33.50	R: 42.78	F1: 36.37
Meteor score: 0.501564144004798
	rouge-1:	P: 51.58	R: 66.46	F1: 56.37
	rouge-2:	P: 40.01	R: 50.65	F1: 43.49
	rouge-3:	P: 36.68	R: 46.55	F1: 39.92
	rouge-4:	P: 35.00	R: 44.47	F1: 38.11
	rouge-l:	P: 36.78	R: 47.07	F1: 4

KeyboardInterrupt: 